With the rise of e-books, reading literature from around the world has become much more accessible, but the language barrier still remains.

This work aims to help for this issue.

We will use the Dracula 1897 edition avaliable on https://www.gutenberg.org/ebooks/345

Project Gutenberg is an online library of free e-books that aims to make classic literature more widely available. It was founded in 1971 by Michael Hart, and is the oldest digital library. The project's mission is to digitize and make available books that are in the public domain, which means that the copyright has expired and the books are no longer under exclusive rights.

Please, this work is just embryonal and, maybe, doesn't solve the translation satisfactorily. In fact, it's just a simple and automatic way to use Google Translate or another translator provider, with means that it will reproduce their deficiencies and issues.

Keep in mind that there is a far difference between translation and localization. Much of the work of a professional translator is not just translating a sentence, but transposing the idea closed on it. And there are writers who usually write their books in layers or profound analogies, while their culture lies in it.

Even an AI or Deep Learning system will need s much more fancy solution for this issue.

So, this algorithm is just a first step but, for a really better result, it is proper that you have a reasonable command of the objective language so as not to incur misunderstandings in your study.

At last, but importantly, each ebook has its own structure, so depending on it, the translation could not perform as expected. A more general solution is coming (I hope so), but, for now, please analyze your ebook format before using this algorithm.


In [ ]:
#IMPORT FRAME

from ebooklib import epub
import pandas as pd
import numpy as np
import os
import translators as ts 
import translators.server as tss


# this ons will be important, trust me
import scipy.stats as stats



In [ ]:
#LOADING FRAME

epub_path=os. getcwd()+"/Dracula - Bram Stoker.epub"
#epub_path=os. getcwd()+"/pg345-images-3.epub"
book = epub.read_epub(epub_path)

#for analyzes reason I produce a Dataset with the structure of the book
chapters=pd.DataFrame(columns=["Chapter", "Type"])
info=[]
for item in book.get_items():
    info.append(item.get_name())
    info.append(item.get_type())
    chapters.loc[len(chapters)]=info
    info=[]

print(chapters)

#taking a look if I took the right document
title = book.get_metadata('DC', 'title')[0][0]
author = book.get_metadata('DC', 'creator')[0][0]
print(f"Title: {title}")
print(f"Author: {author}")

In [ ]:
#MAKING LIST OF CHAPTER
# (see EbookLib Documentatio Release 0.17 page 16 )
ch_list=np.array(chapters.Chapter.iloc[np.where(chapters.Type==9)])
#if you want to start and stop in a specific chapters, please uncomment the line above e adapt it 
#refute=np.array(range(0,8))
#ch_list=ch_list[9:len(ch_list)-1]

#checking the result
print(ch_list)

In [ ]:
# LOADING CHAPTER(S) TO A VARIABLE
import ebooklib
from bs4 import BeautifulSoup
txt_ready=""
document="6844709148999201933_345-h-1.htm.xhtml"

# if you want more than one chapter please ajust the ch_list above and substitute ue document variable
for item in book.get_items():
    if (item.get_type() == ebooklib.ITEM_DOCUMENT) and (item.get_name()== document ):
        soup = BeautifulSoup(item.get_content(), 'html.parser')
        txt_ready = txt_ready + soup.get_text()     
            

In [ ]:
#ANALYZE FRAME#
all_txt=txt_ready.split("\n")
p_size=[]
s_size=[]
pz_count=0
sz_count=0
for paragraph in all_txt:
    p_size.append(len(paragraph))
    sentence=paragraph.split(".")
    if len(paragraph)<3:
        pz_count+=1
    for s in sentence:
        if len(s)<3:
            sz_count+=1
        s_size.append(len(s))
        
print("Description of paragraphs")
print(stats.describe(p_size))
print("There are {} paragraphs without words".format(pz_count))
print("Description of sentences")
print(stats.describe(s_size))
print("There are {} sentences without words".format(sz_count))


In [ ]:
#TRANSLATION FRAME#
from_language, to_language = 'en', 'pt'

all_txt=txt_ready.split("\n")
total=len(txt_ready)
translation=""
max_size=5000 #max size allowed by google translate
flag_dot=False

for paragraph in all_txt:
    size_translation=len(translation)
    size_paragraph=len(paragraph)
    print(" Status{}% translated".format(size_translation/total*100))
    #checking if it's now above the provider max characters size and, at least, that
    #we will translate, at least, one word
    if size_paragraph<=max_size and size_paragraph>3:
        sentence=tss.google(paragraph, from_language, to_language)
        translation = translation + "\n" + sentence 
    #if the  paragraph is larger than the limit asked by the provider
    #we break it in sentences
    elif size_paragraph>max_size:        
        sentence=paragraph.split(".")
        for s in sentence:        
            if len(s)<round(max_size*0.7):
                wait_s=wait_s+s
        #if s starts with a number this means that:
        #1- the sentence starts with a float number
        #2- it's an enumerated sentence
            if int(s[0]):
                wait_s=s
                flag_dot=True
                pass
            if flag_dot:
                lag_dot=False
                s= wait_s+s           
            trans_sentence=tss.google(s, from_language, to_language)
            print(trans_sentence)
            translation = translation + ". " + trans_sentence
        translation=translation+ "\n"
    clear_output()
    

with open("translated_ebook.txt", "w") as file:
    file.write(translation)

file.close()
print("We finished")